<a href="https://colab.research.google.com/github/FCosmin27/ocs/blob/teo/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import random

In [3]:
upper_bound = 100
lower_bound = -100
no_of_variables = 10

In [5]:
def check_x(x):
    for i in range(len(x)):
        if x[i] > upper_bound:
            x[i] = upper_bound
        elif x[i] < lower_bound:
            x[i] = lower_bound
    return x

In [6]:
x = np.random.normal(size=no_of_variables)

x = check_x(x)
print(x)

[-0.87900411 -0.5765651   0.08228428 -0.10521159  1.21151382  1.12903649
  0.61656033  0.90530397 -1.29382545  1.56629841]


In [7]:
def f1(x):
    return np.sum(np.square(x))

In [53]:
def f2(x):
    result = 0
    for i in range(no_of_variables):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result

In [54]:
def f3(x):
    result = 0
    for i in range(1, no_of_variables):
        result += 1e6 ** ((i-1)/(no_of_variables-1)) * x[i]**2

    return result

In [55]:
def f4(x):
    result = 0
    for i in range(no_of_variables):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result * (1 + 0.4 * np.abs(np.random.randn()))

In [11]:
def f5(x):
    A = np.random.randint(-500, 501, size=(no_of_variables, no_of_variables))

    while np.linalg.det(A) == 0:
        A = np.random.randint(-500, 501, size=(no_of_variables, no_of_variables))

    return np.max(np.abs(x*A - A))

In [12]:
print("f1(x):", f1(x))
print("f2(x):", f2(x))
print("f3(x):", f3(x))
print("f4(x):", f4(x))
print("f5(x):", f5(x))

f1(x): 9.192401806586169
f2(x): 16.835277198404732
f3(x): 615998.7376365319
f4(x): 19.262423256165903
f5(x): 1018.458501271515


In [13]:
def calculate_min_x(func, no_of_agents, x, es, alpha):
    min_x = x
    for _ in range(es):
        xm = min_x + alpha * np.random.normal(size=no_of_variables)
        xm = check_x(xm)
        if func(xm) < func(min_x):
            min_x = xm
    return min_x

In [14]:
def get_population(number_of_variables, population_size):
    return np.random.normal(size=(population_size, number_of_variables))

In [45]:
initial_alpha = 0.5
no_of_variables = 10
population_size = 5
iterations = 100
functions = [f1, f2, f3, f4, f5]
population = get_population(no_of_variables, population_size)

# Lab2

In [33]:
def population_v2_adaptive(initial_alpha, population, function, no_of_variables, max_NFE):
    NFE = 0
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for agent in population:
            new_agents = [agent + alpha * np.random.normal(size=no_of_variables) for _ in range(len(population))]
            best_agent = min(new_agents, key=function)
            new_population.append(best_agent)

            NFE += len(population)

            if NFE >= max_NFE:
                break

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

        if NFE >= max_NFE:
            break

    return population

In [56]:
print("f1(x):", population_v2_adaptive(population, f1))
print("f2(x):", population_v2_adaptive(population, f2))
print("f3(x):", population_v2_adaptive(population, f3))
print("f4(x):", population_v2_adaptive(population, f4))
print("f5(x):", population_v2_adaptive(population, f5))

TypeError: population_v2_adaptive() missing 3 required positional arguments: 'function', 'no_of_variables', and 'max_NFE'

In [ ]:
for idx, population in enumerate(population_v2_adaptive()):
    print("Function", idx+1)
    for agent in population:
        print(agent, f1(agent))


In [16]:
def pc_function(iter_no):
    return max(0.5, 1 / (1 + iter_no))

In [17]:
def pm_function(iter_no):
    return max(0.01, 0.1 / (1 + iter_no))

In [18]:
def sphere_function(x):
    return np.sum(np.square(x))

# Lab3

In [34]:
def CGA_adaptiveV1_greedy(pc_function, pm_function, population, function, no_of_variables):
    NFE = 0
    max_NFE = 1000
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        pc = pc_function(iter_no)
        pm = pm_function(iter_no)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, no_of_variables - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population

In [35]:
final_population = CGA_adaptiveV1_greedy(pc_function, pm_function, population, sphere_function, no_of_variables)
print("Best solution (CGA):", min(final_population, key=sphere_function))


Best solution (CGA): [ 0.10849433  0.56144834 -1.21562659 -0.08643343  0.00310515  0.26575802
 -0.15021836  0.20669168  0.04663977  0.47775293]


In [19]:
def pc_function_fitness(fitness):
    return 0.5 + 0.4 * fitness

In [20]:
def pm_function_fitness(fitness):
    return 0.1 - 0.08 * fitness

# Lab4

In [ ]:
def RGA_4(population, function, pc=0.9, pm=0.1, max_NFE=1000):
    NFE = 0
    no_of_variables = population.shape[1]
    while NFE < max_NFE:
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = population[np.random.choice(len(population), 2, replace=False)]

            if np.random.rand() < pc:
                alpha = 0.1
                child1 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
                child2 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child1[mutation_idx] += np.random.normal()
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child2[mutation_idx] += np.random.normal()

            child1 = np.clip(child1, -10, 10)
            child2 = np.clip(child2, -10, 10)

            new_population.extend([child1, child2])

        combined_population = np.vstack((population, new_population))
        combined_population = sorted(combined_population, key=function)
        population = np.array(combined_population[:len(population)])

        NFE += len(new_population)

    return population

In [ ]:
final_population_v2 = RGA_4(population, f1)
print("Best solution (RGA_4):", min(final_population_v2, key=f1))

Best solution (CGA V2): [-3.05743887e-02  6.22209232e-02 -2.99545163e-02  1.16265863e-02
  6.75334746e-03 -8.92766834e-03  2.83640722e-02 -3.70288975e-04
 -1.44694861e-04  3.59688935e-01]


# Lab5

In [ ]:
def DE_best_1_exp(population, function, F=0.8, CR=0.9, max_NFE=1000):
    NFE = 0
    population_size, no_of_variables = population.shape

    while NFE < max_NFE:
        new_population = []

        for i in range(population_size):
            best = min(population, key=function)
            r1, r2 = population[np.random.choice(population_size, 2, replace=False)]
            mutant = best + F * (r1 - r2)

            trial = np.copy(population[i])
            start_idx = np.random.randint(no_of_variables)
            for j in range(no_of_variables):
                if np.random.rand() < CR or j == start_idx:
                    trial[j] = mutant[j]

            trial = np.clip(trial, -10, 10)

            if function(trial) < function(population[i]):
                new_population.append(trial)
            else:
                new_population.append(population[i])

        population = np.array(new_population)
        NFE += population_size

    return population


In [ ]:
final_population_v2 = DE_best_1_exp(population, f1)
print("Best solution (DE_best_1_exp):", min(final_population_v2, key=f1))

Best solution (DE_best_1_exp): [-0.23885957  0.221679    0.14638907 -0.26745006  0.19292714  0.03546124
 -0.30634016 -0.19258842 -0.00337336  0.19196741]


## Butnaru Teodor

# Lab2

In [57]:
def population_v1_self_adaptive(population, function, dimension = 10, max_NFE = 1000):
    NFE = 0
    iter_no = 0
    alfa = [[30, -30] for _ in range(len(population))]

    while NFE < max_NFE:
        iter_no += 1
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for i, agent in enumerate(population):
            xm = [
                agent[0] + alfa[i][0] * random.uniform(-1, 1),
                agent[1] + alfa[i][1] * random.uniform(-1, 1)
            ]

            if function(agent) > function(xm):
                new_population.append(xm)
            else:
                new_population.append(agent)

            alfa[i][0] += alfa[i][0] * random.uniform(-1, 1)
            alfa[i][1] += alfa[i][1] * random.uniform(-1, 1)

            NFE += len(population)

            if NFE >= max_NFE:
                break

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

        if NFE >= max_NFE:
            break

    return population

In [58]:
print("f1(x):", population_v1_self_adaptive(population, f1))
print("f2(x):", population_v1_self_adaptive(population, f2))
print("f3(x):", population_v1_self_adaptive(population, f3))
print("f4(x):", population_v1_self_adaptive(population, f4))
print("f5(x):", population_v1_self_adaptive(population, f5))

f1(x): [[-0.028707077560294347, -0.40171296443427756], [-0.4460398638032624, -1.9251323360092654], [-0.2676028936138475, 0.5069977458802547], [-0.8665317398598364, -0.49021084154201466], [0.013984910477110202, -0.5985829750532308]]
f2(x): [[0.0034978510649388405, -53.28279388614703], [-0.001555756369914631, 2.051802786658235], [-0.721201471043054, 0.5275964633239242], [10.576728520633068, -413.83455117209303], [-0.006670606245770325, 27.421584489089962]]
f3(x): [[-21.596078937035184, -0.07622574081365761], [30.765471785182676, -5.05059929683791e-06], [1.4247516408662964, 0.08199798308355533], [14.261591768961258, -0.7839936501665126], [10.450195144895657, 0.038789596103839495]]
f4(x): [[-0.021141690006162867, -0.39905336680017495], [0.0527092615974961, -30.361027632549703], [0.7275174143929293, 0.5401314724802637], [0.22771467686145377, -69.61045491831666], [3.159967332268124, 15.765115744937743]]


ValueError: operands could not be broadcast together with shapes (2,) (10,10) 

# Lab3

In [41]:
def pc_function_fit(avg_fitness):
    return max(0.5, 1 / (1 + avg_fitness))

def pm_function_fit(avg_fitness):
    return min(0.1, 1 / (1 + avg_fitness))

def sphere_function_fit(agent):
    return sum(x**2 for x in agent)

In [40]:
def CGA_adaptiveV2_greedy(population, function ,dimension, pc_function = pc_function_fit, pm_function = pm_function_fit, max_NFE = 1000):
    NFE = 0
    max_NFE = 1000
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1

        # Calculate fitness-based probabilities (pc and pm) based on the population
        avg_fitness = np.mean([function(agent) for agent in population])

        # Adjust pc and pm based on average fitness
        pc = pc_function_fit(avg_fitness)
        pm = pm_function_fit(avg_fitness)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            # Crossover
            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, dimension - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            # Mutation
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        # Sort the new population based on fitness (function value)
        new_population.sort(key=function)

        # Preserve the best solution (elitism)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent  # Replace worst solution with best solution

        # Update population
        population = new_population

        # Update NFE count (each agent's fitness is evaluated once)
        NFE += len(new_population)

    return population

In [42]:
final_population = CGA_adaptiveV2_greedy(pc_function, pm_function, population, sphere_function_fit, no_of_variables)
print("Best solution (CGA):", min(final_population, key=sphere_function))

Best solution (CGA): [-0.14124297  0.04880993 -0.12926254 -0.42057245  0.00310515  0.14709243
  0.02929361  0.01432609 -0.30452415  0.18803122]


# Lab4

In [22]:
def RGA_3(population, function, dimension, pc = 0.8, pm = 0.1, max_NFE = 1000):
    NFE = 0

    while NFE < max_NFE:

        fitness_values = np.array([function(individual) for individual in population])
        NFE += len(population)

        new_population = []

        for _ in range(len(population) // 2):

            parent1 = population[np.random.randint(len(population))]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                d = np.abs(parent1 - parent2)
                lower = np.minimum(parent1, parent2) - 0.5 * d
                upper = np.maximum(parent1, parent2) + 0.5 * d
                child1 = np.random.uniform(lower, upper)
                child2 = np.random.uniform(lower, upper)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child1[mutation_idx] += np.random.normal()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child2[mutation_idx] += np.random.normal()

            new_population.extend([child1, child2])

        new_population.sort(key=function)
        population = new_population[:len(population)]

    best_solution = min(population, key=function)
    return best_solution, function(best_solution), NFE


In [24]:

best_solution, best_fitness, nfe_used = RGA_3(
    pc=0.8,
    pm=0.1,
    population=population,
    function=f1,
    dimension=no_of_variables
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

Best Solution: [ 0.76532856  1.03264963 -1.77842379  0.79206066  0.19735348  2.13655596
 -1.89392779  0.19496101  0.95186293 -0.75241319]
Best Fitness: 15.143204964954823
Number of Function Evaluations: 1001


# Lab5

In [51]:
def DE_rand_to_best_1_exp(population, function, dimension, F = 0.5, CR = 0.9,  max_NFE = 1000):
    NFE = 0
    population_size = len(population)

    while NFE < max_NFE:
        fitness_values = np.array([function(ind) for ind in population])
        best_idx = np.argmin(fitness_values)
        best_individual = population[best_idx]
        NFE += len(population)

        new_population = []

        for i in range(population_size):
            x_rand = population[np.random.randint(population_size)]
            x1, x2 = population[np.random.choice([j for j in range(population_size) if j != i], size=2, replace=False)]
            mutant_vector = x_rand + F * (best_individual - x_rand) + F * (x1 - x2)

            trial_vector = population[i].copy()
            start_index = np.random.randint(dimension)
            length = 0
            while np.random.rand() < CR and length < dimension:
                trial_vector[(start_index + length) % dimension] = mutant_vector[(start_index + length) % dimension]
                length += 1

            if function(trial_vector) < fitness_values[i]:
                new_population.append(trial_vector)
            else:
                new_population.append(population[i])

        population = np.array(new_population)

    final_fitness_values = np.array([function(ind) for ind in population])
    best_idx = np.argmin(final_fitness_values)
    return population[best_idx], final_fitness_values[best_idx], NFE


In [52]:
best_solution, best_fitness, nfe_used = DE_rand_to_best_1_exp(
    F=0.5,
    CR=0.9,
    population=population,
    function=f2,
    dimension=no_of_variables
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

Best Solution: [ 0.25040852  0.20740308  0.3478378  -0.15882257 -0.90409665 -0.47458761
 -0.08317977  0.86823301 -0.29076429  0.04735704]
Best Fitness: 2.6651090402251234
Number of Function Evaluations: 1000


**Laboratory 6. Testing meta-heuristics**

In [ ]:
D = [5, 10]
NEF = 1000

functions =